<a href="https://colab.research.google.com/github/sunnyna/TIL/blob/master/bond_analysis_report_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

drive 연결

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 채권 분석 리포트 크롤링

In [3]:
import requests
from bs4 import BeautifulSoup

pdf_list = []
date_list = []
title_list = []

for i in range(23,127):
  url = 'https://finance.naver.com/research/debenture_list.nhn?'
  params = {
      'page' : i
  }

  resp = requests.get(url,params=params)
  soup = BeautifulSoup(resp.content, 'html.parser')

  td_tag = soup.findAll("td",{"class":"file"})
  date_tag = soup.findAll('td', {"style":"padding-left:5px"} )
  title_tag = soup.findAll('td', {"style":"padding-left:10px"})



  for t in td_tag:
    pdf_url=t.find('a').attrs['href']
    pdf_list.append(pdf_url)



  for date in date_tag:
    date_list.append(date.get_text())

  for title in title_tag:
    title_list.append(title.get_text())


갯수 확인 및 슬라이싱(08.04.01~17.12.27까지만)

In [4]:
print(len(pdf_list))   
print(len(date_list))
print(len(title_list))


print(date_list[-10:])
print(date_list[:10])

3099
3099
3099
['08.05.20', '08.05.19', '08.05.13', '08.05.04', '08.04.28', '08.04.21', '08.04.14', '08.04.07', '08.04.07', '08.04.01']
['18.01.02', '18.01.02', '17.12.27', '17.12.26', '17.12.20', '17.12.20', '17.12.18', '17.12.15', '17.12.15', '17.12.14']


슬라이싱 된 것 확인

In [5]:

pdf_list_ = pdf_list[2:]
date_list_ = date_list[2:]
title_list_ = title_list[2:]

print(date_list_[-10:])
print(date_list_[:10])

['08.05.20', '08.05.19', '08.05.13', '08.05.04', '08.04.28', '08.04.21', '08.04.14', '08.04.07', '08.04.07', '08.04.01']
['17.12.27', '17.12.26', '17.12.20', '17.12.20', '17.12.18', '17.12.15', '17.12.15', '17.12.14', '17.12.14', '17.12.13']


In [6]:
print(len(pdf_list_))
print(len(date_list_))
print(len(title_list_))


3097
3097
3097


/ 가 경로오류가 생겨서 전처리해줌

In [ ]:
title_list__ = []
for t in title_list :
  title_list__.append(t.replace('/', ','))

title_list__

#해외크레딧/신흥국 국채 : 만기 짧은 미국 회사채에 ..' 
# /때문에 경로오류가생김

### pdf 파일로 저장
path 주의해서 확인하기

In [8]:

for pdf,title,date in zip(pdf_list_, title_list__, date_list_ ):
  with open('/content/drive/My Drive/bond_analysis_report/{0}_{1}.pdf'.format(date,title), "wb") as file:
    response = requests.get(pdf)
    file.write(response.content)

파일갯수가 리스트 갯수와 일치하는 지 확인 (겹치는 제목은 덮어쓰기 때문에 제대로 나왔는지 확인해야했음)

In [10]:
import os

path = "/content/drive/My Drive/bond_analysis_report"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

len(file_list)

file_list: ['17.12.27_2018년 북한 리스크 대비하기.pdf', '17.12.26_1월에는 무조건 GO.pdf', '17.12.20_트럼프노믹스를 극복한 2017년 시장 .pdf', '17.12.20_11월 채권시장 전망.pdf', '17.12.18_2금융권 안심전환대출 수급영향 점검 .pdf', '17.12.15_확신 없는 인상의견도 있었음이 확인.pdf', '17.12.15_속도 조절 중.pdf', '17.12.14_성장과 물가의 Dilemma.pdf', '17.12.14_12월 FOMC 함의와 2018년 연준구성 변화 .pdf', '17.12.13_1사분기 통화정책 불확실성 확대 가능성 경계.pdf', '17.12.08_금리를 인상한 연준 2018년은 어떨까?.pdf', '17.12.07_이제는 집에 가야 할 시간 .pdf', '17.12.06_연말 장세에 돌입.pdf', '17.12.04_브라질 기준금리, 50bp 인하한 7.00% .pdf', '17.12.04_2018년 트렌드 점검 (1) 한-미 금리 역전 본격화.pdf', '17.12.01_중단기물 중심의 강세 이어질 전망.pdf', '17.12.01_변동성에 약한 크레딧시장.pdf', '17.12.01_11월 금통위 Review 신중한 금리인상 .pdf', '17.12.01_금리인상의 기다ᄅ

3093

# pdf를 csv로 변환

In [11]:
!pip install tika
from tika import parser


1. pdf파일을 txt로 변환하는 함수정의

In [20]:
def convert_pdf_to_txt(file_name):
  rawText = parser.from_file('{}'.format(file_name))
  rawData = rawText['content']

  if rawData is not None:
    global rawData_ 
    rawData_ = rawData.replace('\n','') 


  return rawData_

'NoneType' object has no attribute 'replace' 오류수정  
None 일때 예외를 줘야됨

파일이름과 내용으로 이루어진 csv 파일 생성

In [21]:
import csv
from pathlib import Path

folder = Path('/content/drive/My Drive/bond_analysis_report')
csv_file = Path('/content/drive/My Drive/bond_analysis_report_list.csv')

with csv_file.open('w', encoding='utf-8') as f:
    writer = csv.writer(f, csv.QUOTE_ALL)
    writer.writerow(['FileName', 'Text'])
    for pdf_file in folder.glob('*.pdf'):
        pdf_text = convert_pdf_to_txt(pdf_file)
        writer.writerow([pdf_file.name, pdf_text])


csv 파일을 df로 저장

In [22]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/bond_analysis_report_list.csv')
df

,FileName,Text
0,17.12.27_2018년 북한 리스크 대비하기.pdf,Microsoft Word - CrediVille_171227 트럼...
1,17.12.26_1월에는 무조건 GO.pdf,PowerPoint 프레젠테이션Fixed Income Analyst 김지만 02-3...
2,17.12.20_트럼프노믹스를 극복한 2017년 ᄉ...,2013년 0월 0일 2금융권 안심전환대출 수급영향 점검 Inve...
3,17.12.20_11월 채권시장 전망.pdf,Fixed Income 2017. 12. 20 11월 금통위 의...
4,17.12.18_2금융권 안심전환대출 수급영...,(Microsoft Word - \307\366\264\353\302\367_Cre...
...,...,...
3088,08.04.21_정부의 금리인하 요구에 한은의 부...,Microsoft Word - 920_0421_Fixed Income Weekly....
3089,08.04.14_FOMC 이후 원자재 가격 상승세 둔ᄒ...,Microsoft Word - Fixedincome0414.doc 4월 금통위...
3090,08.04.07_항상 열려는 있는 가능성.pdf,Microsoft Word - 0407.docFixed Income Weekly 2...
3091,08.04.07_총재의 변신은 무죄다?!.pdf,Microsoft Word - 0407.docFixed Income Weekly 2...
